In [4]:
# Import necessary libraries
import os, requests, db_utils, time
import pandas as pd
import numpy as np
connect = db_utils.get_connection()
import db_utils, sqlalchemy
import pandas as pd
import numpy as np
import db_utils, sqlalchemy
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import seaborn as sns
from scipy.stats import fisher_exact
from scipy import stats
import math
import os
import gzip
import re
import glob
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import math

In [5]:
# Define a function that creates instances representing specific PDB IDs with associated chain information
class PDB_ID:
    def __init__(self,pdb_id):
        self.pdb_id = pdb_id
        self.chain_uniprot_id = {}
        
    def __repr__(self):
        line = f'{self.pdb_id} has the following chain information:\n'
        for k, v in self.chain_uniprot_id.items():
            line += f'chain {k} with protein {v}\n'
        return line

In [6]:
# Define a generator function that yields chunks of elements from a given list based on a specified chunk size
def yield_elements(lst, chunk_size):
    """
    Yields specified size of elements from a list.

    Args:
    - lst (list): The list from which elements are yielded.
    - chunk_size (int): The size of chunks to yield.

    Yields:
    - list: A chunk of elements.
    """
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

In [7]:
# Define a function that retrieves Uniprot IDs associated with specified PDB IDs using the RCSB PDB GraphQL API.
def retrieve_pdb_id_uniprot(pdb_id_list):
    pdb_ids = ','.join([f'\"{pdb_id}\"' for pdb_id in pdb_id_list])
    url = "https://data.rcsb.org/graphql"
    query="""{
      entries(entry_ids:[%s]) {
        polymer_entities {
          rcsb_polymer_entity_container_identifiers {
            entry_id
            auth_asym_ids
          }
          uniprots {
            rcsb_uniprot_container_identifiers {
              uniprot_id
            }
          }
        }
      }
    }"""% pdb_ids
    response = requests.post(url, json={'query': query})

    # Check the response
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
    return data


# Define a function that processes RCSB PDB GraphQL API data and updates a global dictionary (PDB_ID_dict)
# with information about PDB IDs, their associated chain IDs, and corresponding Uniprot IDs
def read_response_into_PDB_ID(response_data):
    global PDB_ID_dict
    info = response_data.get("data")
    for entry in info.get('entries'):
        polymer_entities = entry.get('polymer_entities')
        for polymer_entity in polymer_entities:
            pdb_id = polymer_entity.get("rcsb_polymer_entity_container_identifiers").get('entry_id')
            if pdb_id not in PDB_ID_dict:
                pdb_obj = PDB_ID(pdb_id)
            else:
                pdb_obj = PDB_ID_dict.get(pdb_id)
            chain_ids = polymer_entity.get("rcsb_polymer_entity_container_identifiers").get('auth_asym_ids')
            for chain in chain_ids:
                pdb_obj.chain_uniprot_id[chain] = []
            uniprots = polymer_entity.get('uniprots')
            if uniprots is None:
                uniprot_id = np.nan
                for chain in chain_ids:
                    pdb_obj.chain_uniprot_id[chain].append(uniprot_id)
            else:
                for uniprot in uniprots:
                    uniprot_id = uniprot.get('rcsb_uniprot_container_identifiers').get('uniprot_id')

                    for chain in chain_ids:
                        pdb_obj.chain_uniprot_id[chain].append(uniprot_id)
            PDB_ID_dict[pdb_id] = pdb_obj

In [8]:
# Query MySQL to get all distinct pdb_ids in 3did
query = """select distinct PDB_ID from chopyan_db.DDI_crystal_structure_features_complete;"""

with connect.cursor() as cursor:
    cursor.execute(query)
    pdb_ids = cursor.fetchall()

pdb_ids = [e.upper() for ele in pdb_ids for e in ele]
len(pdb_ids)

100760

In [9]:
# Make a dictionary to contain the PDB_ID instances
PDB_ID_dict = {}

for chunk in yield_elements(pdb_ids, 1000):
    response_data = retrieve_pdb_id_uniprot(chunk)
    read_response_into_PDB_ID(response_data)
    print('Sleeping 1 second to not flood the server')
    time.sleep(1)

Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the server
Sleeping 1 second to not flood the

In [10]:
# Connect to MySQL and read the DDI types in 3did along with their chain information in all supporting structures
# (which chains in a structure hold the two domains forming a DDI) into a DataFrame
# Obtain all chain IDs of proteins carrying a domain forming a 3did DDI type for all PDB structures in 3did
eng = sqlalchemy.create_engine('mysql://', creator= db_utils.get_connection)
query= pd.read_sql_query('''select DDI_type, PDB_ID, IsInterchain, Chain1, Chain2 from chopyan_db.DDI_crystal_structure_features_complete as df''', con= eng)
df= pd.DataFrame(query)
df['HomoDDI'] = [1 if ele.split('_')[0] == ele.split('_')[1] else 0 for ele in df['DDI_type']]
df

,DDI_type,PDB_ID,IsInterchain,Chain1,Chain2,HomoDDI
0,PF00001_PF00001,1GZM,1,A,B,1
1,PF00001_PF00001,1HZX,0,B,B,1
2,PF00001_PF00001,1L9H,0,B,B,1
3,PF00001_PF00001,2J4Y,1,A,B,1
4,PF00001_PF00001,2VT4,1,C,D,1
...,...,...,...,...,...,...
235443,PF18841_PF17433,1WMR,0,A,A,0
235444,PF18841_PF17433,1X0C,0,B,B,0
235445,PF18841_PF17433,2Z8G,0,A,A,0
235446,PF18841_PF17433,3WWG,0,C,C,0


In [11]:
# Derive the respective UniProt ID for all proteins in the 3did dataset carrying a domain forming a DDI type
# Map the chain IDs in the 3did dataset to their UniProt IDs and add that information to the 3did DataFrame
UniProt1 = []
for PDB_ID, chain_ID in zip(df['PDB_ID'], df['Chain1']):
    if PDB_ID in PDB_ID_dict:
        #print(PDB_ID_dict.get(PDB_ID).chain_uniprot_id.get(chain_ID))
        if PDB_ID_dict.get(PDB_ID).chain_uniprot_id.get(chain_ID) is None:
            UniProt1.append('No UniProt')
        elif type(PDB_ID_dict.get(PDB_ID).chain_uniprot_id.get(chain_ID)[0]) == float:
            UniProt1.append('No UniProt')
        else:
            #print(PDB_ID_dict.get(PDB_ID).chain_uniprot_id.get(chain_ID))
            UniProt1.append(','.join(PDB_ID_dict.get(PDB_ID).chain_uniprot_id.get(chain_ID)))
    else:
        UniProt1.append('No UniProt')
        
UniProt2 = []
for PDB_ID, chain_ID in zip(df['PDB_ID'], df['Chain2']):
    if PDB_ID in PDB_ID_dict:
        #print(PDB_ID_dict.get(PDB_ID).chain_uniprot_id.get(chain_ID))
        if PDB_ID_dict.get(PDB_ID).chain_uniprot_id.get(chain_ID) is None:
            UniProt2.append('No UniProt')
        elif type(PDB_ID_dict.get(PDB_ID).chain_uniprot_id.get(chain_ID)[0]) == float:
            UniProt2.append('No UniProt')
        else:
            #print(PDB_ID_dict.get(PDB_ID).chain_uniprot_id.get(chain_ID))
            UniProt2.append(','.join(PDB_ID_dict.get(PDB_ID).chain_uniprot_id.get(chain_ID)))
    else:
        UniProt2.append('No UniProt')

In [12]:
df['UniProt1'] = UniProt1
df['UniProt2'] = UniProt2

In [13]:
df['HomoProtein'] = np.where(df['UniProt1']== df['UniProt2'], 1,0)

In [44]:
df

,DDI_type,PDB_ID,IsInterchain,Chain1,Chain2,HomoDDI,UniProt1,UniProt2,HomoProtein
0,PF00001_PF00001,1GZM,1,A,B,1,P02699,P02699,1
1,PF00001_PF00001,1HZX,0,B,B,1,P02699,P02699,1
2,PF00001_PF00001,1L9H,0,B,B,1,P02699,P02699,1
3,PF00001_PF00001,2J4Y,1,A,B,1,P02699,P02699,1
4,PF00001_PF00001,2VT4,1,C,D,1,P07700,P07700,1
...,...,...,...,...,...,...,...,...,...
235443,PF18841_PF17433,1WMR,0,A,A,0,O00105,O00105,1
235444,PF18841_PF17433,1X0C,0,B,B,0,O00105,O00105,1
235445,PF18841_PF17433,2Z8G,0,A,A,0,O00105,O00105,1
235446,PF18841_PF17433,3WWG,0,C,C,0,O00105,O00105,1


In [45]:
df.to_csv('/Users/johgeist/Downloads/DDI_type_structure_level_interchain_homo_protein_whole_3did.csv')

In [14]:
# Group the DataFrame based on 'DDI_type' to categorize the DDI types in 3did as only supported by only intrachain
# interfaces or supported by at least one interchain interface
grouped = df.groupby('DDI_type').agg({'HomoProtein': min, 'IsInterchain': max, 'HomoDDI': 'first', 'UniProt1': 'first', 'UniProt2': 'first'}).reset_index()

In [17]:
grouped.to_csv('/Users/johgeist/Downloads/DDI_type_interchain_homo_protein_whole_3did.csv')

In [18]:
grouped.groupby(['IsInterchain', 'HomoDDI','HomoProtein']).size()

IsInterchain  HomoDDI  HomoProtein
0             0        1              1970
              1        1               250
1             0        0              5440
                       1              1713
              1        0               284
                       1              5315
dtype: int64